# Harmonic Oscillator

A simple harmonic oscillator is an oscillator that is neighther driven nor damped. It consists of mass `m` which experiences a single force `F` which pulls the mass in the direction of teh point `x=0` and depends only on the position `x` of the mass and a constant `k` 

![](https://lh3.googleusercontent.com/proxy/Cp7rgoxGR2JHqv0-ZgQWyDc9ZTWRCHR11t2kuIzHlY2toBcqRoZg8KqeqW7iCThye1K7n4igDHRj8N-B9-vdLL5WfXID-PF3lGagiXxFWIC2cV1qwzupdPo-wWt-ihwAzB31x1FMDwNGkZ_RslDxCydzve-ALI7vCMr2dQfyAbYqvGVFPrBH5O8)

#### **Framing equations**:
Using Newtons second law

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e700293120d5c492003ea0f6fc075e23fe54481a)

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e98b4f368cf0f8db5deae9c10c2423ab28d2e6f8)

#### **ODE**

$$\dot{x} = v$$

$$ \dot{v} = - \frac{k}{m}x$$

#### forward Euler method

$$ x_{n+1} = x_{n} + dt*(v_{n}) $$
$$  v_{n+1} = v_{n} + dt*(- \frac{k}{m}x_{n})  $$

# Solving Numerically

### 1. Import LIbraries

In [1]:
import numpy as np
import pandas as pd
from numba import jit
from math import sin,cos, sqrt, fabs

/nethome/n.kusampudi/.local/lib/python3.6/site-packages/numba/core/errors.py:154: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
#plotting libraries
import datashader as ds
from datashader import transfer_functions as tf
from datashader.colors import inferno, viridis
from colorcet import palette
import holoviews as hv
from holoviews.operation.datashader import datashade, dynspread

### 2. Define ODE

In [ ]:
@jit(nopython=True)
def simp_Harm_osc(x,v,k,m,dt,*o):
    """
    returns the next values of the iteration 
    x,v: system varaibles
    k,m: system parameters
    dt: delta time
    """
    return x + dt*(v),\
            v - dt*((k/m)*x)


### 3. Function to get trajectory

In [ ]:
n = 100 # number of iterations
@jit(nopython=True)
def trajectory_coords(fn,x0,v0,k,m,dt,n=n):
    """
    returns trajectory of given ODE for n iterations
    x0,y0: initial position, velocity
    k,m: stiffness, mass
    dt,n: delta_time, total no of iterations
    """
    # create empty arrays
    x,v = np.zeros(n), np.zeros(n)
    
    # assign inital conditions
    x[0], v[0] = x0,v0
    
    # get trajectory (main loop)
    for i in np.arange(n-1):
        x[i+1], v[i+1] = fn(x[i], v[i],k,m,dt)
    return x,v

In [ ]:
def trajectory(fn,x0,v0,k,m,dt,n=n):
    """
    returns the trajectory as pandas dataframe
    """
    x,v = trajectory_coords(fn,x0,v0,k,m,dt,n=n)
    
    return pd.DataFrame(dict(x=x,v=v))

In [ ]:
def get_attractor()

### 4.  Test simulations

#### get data

In [ ]:
# test
test_df = trajectory(simp_Harm_osc, 1.0,1.0,2.0,3.0,0.01,1000000)

In [ ]:
test_df.tail()

#### plot attractor

In [ ]:
palette["viridis"] = viridis
palette["inferno"] = inferno

In [ ]:
# plotting trajectory
ds.transfer_functions.Image.border=0
cvs = ds.Canvas(plot_width=300, plot_height=300)
agg = cvs.points(test_df, "x","v")
tf.shade(agg, cmap=palette["viridis"])

In [ ]:
# Bokeh plot
hv.extension("bokeh")
dynspread(datashade(hv.Points(test_df),
                   cmap= "viridis").opts(width=400,height=400))

In [ ]:
# interactive bokeh

In [3]:
def interactive_dynamics(x0,v0,k,m,dt,n=100):
    
    """
    function that plots trajectory
    """
    return datashade(hv.Points(trajectory(simp_Harm_osc,x0,v0,k,m,dt,n=n)),
                    cmap=inferno[::-1],dynamic=False)

In [4]:
x0,v0,k,m,dt,n = 1.0,1.0,1.0,1.0,0.01,100

In [5]:
dm = hv.DynamicMap(interactive_dynamics, kdims=["x0","v0","k","m","dt","n"])
dm = dm.redim.range(x0=(-2.0,2.0),v0=(-2.0,2.0),k=(-2.0,2.0),m=(-2.0,2.0),
                    dt=(0.01,10.0),n=(100,1000))
dm = dm.redim.default(x0=x0,v0=v0,k=k,
                      m=m,dt=dt,n=n).opts(width=400,height=400)
dm

:DynamicMap   [x0,v0,k,m,dt,n]